In [4]:
import pandas as pd
import numpy as np
from nsepy import get_history
import datetime
import matplotlib
%matplotlib inline

In [5]:
from scipy import stats

In [6]:
%load_ext autoreload
%autoreload 2

# Fetching Daily Returns

In [1]:
def get_returns_from_close_as_dataframe(nse_tickers, start_date, end_date):
    """
    Input: list of tickers for which nse data is needed, start date and end date of period.
    Output: two dictionaries with tickers as keys and daily closing prices and daily returns as dataframes
    """
    closing_prices = {}
    returns = {}
    for ticker in nse_tickers:
        print(ticker)
        data = get_history(symbol=ticker, start=start_date, end=end_date)
        print('done :)')
        closing_prices_df = pd.DataFrame(index=pd.to_datetime(list(data.index), format="%Y-%m-%d"))
        returns_df = pd.DataFrame(index=pd.to_datetime(list(data.index), format="%Y-%m-%d"))
        closing_prices_df[ticker] = data['Close'].to_list()
        returns_df = closing_prices_df.pct_change().drop(list(closing_prices_df.index)[0])
        closing_prices[ticker], returns[ticker] = closing_prices_df, returns_df
    return closing_prices, returns

In [ ]:
## Fetching Daily Returns from NSE and saving as a dictionary of dataframes 
start_date = datetime.date(2009,1,1)
end_date = datetime.date.today()
nse_tickers = ["AVANTIFEED","BAJAJFINSV","BAJAJHLDNG","BAJFINANCE"]
closing_prices, returns = get_returns_from_close_as_dataframe(nse_tickers,start_date,end_date)

# Annualized Returns, Volatility, Return to Risk Ratio and Sharpe Ratio

In [47]:
def get_AnnVol_AnnRet_Ret2Risk_Sharpe(returns, annualizing_factor=260, risk_free_return=0.0581):
    """
    Input: a dataframe of returns, the annualizing factor depending on the frequency of returns, risk free return
    Output: annualised_vol, annualized_return, return_2_risk ratio, sharpe_ratio
    """
    stats_df = pd.DataFrame()
    for returns_df in returns.values():
        annualised_vol = returns_df.std()*np.sqrt(annualizing_factor)
        annualized_return = (returns_df+1).prod()**(annualizing_factor/returns_df.shape[0]) -1
        return_2_risk = annualized_return/annualised_vol
        sharpe_ratio = (annualized_return - risk_free_return)/annualised_vol
        stats_df = pd.concat([stats_df,pd.concat([annualised_vol, annualized_return, return_2_risk, sharpe_ratio], axis="columns")], axis=0)
    stats_df.columns = ['Annualized Volatility','Annualized Returns','Return to Risk Ratio','Sharpe Ratio']
    return stats_df

In [48]:
get_AnnVol_AnnRet_Ret2Risk_Sharpe(returns, 260)

,Annualized Volatility,Annualized Returns,Return to Risk Ratio,Sharpe Ratio
AVANTIFEED,0.693264,-0.239864,-0.345992,-0.429798
BAJAJFINSV,0.376065,0.385105,1.024038,0.869543
BAJAJHLDNG,0.316600,0.236275,0.746289,0.562777
BAJFINANCE,0.494849,0.395548,0.799329,0.681920


# Maximum Drawdown

In [70]:
def get_Drawdown(returns):
    """
    Takes a time series of asset returns.
    returns a DataFrame with columns for the wealth index, 
    the previous peaks, and the percentage drawdown
    """
    drawdowns = pd.DataFrame(index=list(returns.keys()))
    drawdown_value = []
    drawdown_date = []
    for returns_df in returns.values():
        max_drawdown_df = pd.DataFrame(index=list(returns_df.index))
        max_drawdown_df['Wealth Index'] = 1000*(1+returns_df).cumprod()
        max_drawdown_df['Previous Peaks'] = max_drawdown_df['Wealth Index'].cummax()
        max_drawdown_df['Max Drawdown'] = (max_drawdown_df['Wealth Index'] - max_drawdown_df['Previous Peaks'])/max_drawdown_df['Previous Peaks']
        drawdown_value.append(max_drawdown_df['Max Drawdown'].min()*100)
        drawdown_date.append(max_drawdown_df['Max Drawdown'].idxmin())
    drawdowns['Max Drawdown in %'], drawdowns['Max Drawdown Date'] = [drawdown_value, drawdown_date]
    return drawdowns

In [67]:
get_Drawdown(returns)

,Drawdown in %,Drawdown Date
AVANTIFEED,-92.176570,2020-03-23
BAJAJFINSV,-58.589585,2020-05-26
BAJAJHLDNG,-60.922175,2020-03-24
BAJFINANCE,-93.282017,2016-12-22


# Skewness, Kurtosis and Deviation from Normal Distribution

In [23]:
def get_skewness_kurtosis_isnormallydist(returns, level=0.01):
    """
    INPUT: 
    The dictionary with all the returns 
    Required level of confidence that the returns are normally distributed (by default 0.01 or 1%)
    
    OUTPUT:
    Dataframe with 'Returns Mean','Returns Median','Skewed?',
    'Skewness','Excess Kurtosis','Normal as per Jarque?' 
    for each company.
    
    NOTES: 
    A distribution is negatively skewed if the median return (most probable) is less than the average return.
    This is because in normal distribution the mean is equal to the median.
    KURTOSIS FOR NORMALLY DISTRIBUTED RETURNS IS 3. Excess Kurtosis is the observed kurtosis minus 3.
    
    Applies the Jarque-Bera test to determine if a Returns are 
    normal or not Test is applied at the 1% level by default
    Returns True if the hypothesis of normality is accepted, False otherwise
    """
    skewness_df = pd.DataFrame()
    jarque_test = []
    for returns_df in returns.values():
        demeaned_ret = returns_df-returns_df.mean()
        stdev = returns_df.std(ddof=0)      ##Delta Degrees of Freedom. The divisor used in calculations is N - ddof, where N represents the number of elements.
        skewness = ((demeaned_ret**3).mean())/(stdev**3)
        kurtosis = ((demeaned_ret**4).mean())/(stdev**4) -3
        jarque_test.append(stats.jarque_bera(returns_df)[1]>level)
        skewness_df = pd.concat([skewness_df,pd.concat([returns_df.mean(), returns_df.median(), returns_df.mean()>returns_df.median(),skewness,kurtosis], axis="columns")],axis=0)
    skewness_df.columns = ['Returns Mean','Returns Median','Negatively Skewed?','Skewness','Excess Kurtosis']
    skewness_df['Normal as per Jarque?'] = jarque_test
    return skewness_df

In [24]:
get_skewness_kurtosis_isnormallydist(returns)

,Returns Mean,Returns Median,Negatively Skewed?,Skewness,Excess Kurtosis,Normal as per Jarque?
AVANTIFEED,0.000369,-0.001186,True,-7.019527,128.512441,False
BAJAJFINSV,0.001526,0.000187,True,0.140768,10.260906,False
BAJAJHLDNG,0.001009,0.000080,True,0.202100,12.637404,False
BAJFINANCE,0.002097,0.000891,True,-8.468806,264.814673,False


# Semi-Deviation, VaR and Conditional VaR

In [40]:
def get_semideviation_var_cvar(returns, worst_percent=5):
    """
    INPUT:
    the dictionary with returns
    WORST_PERCENT is the percent of most negative returns used to get VaRs
    i.e. returns the number such that that percent of the returns
    fall below the VAR number, and the (100-level) percent are above
    
    OUTPUT:
    dataframe with the semideviation, Historical VaR, Gaussian VaR and CVaR for each return series.
    
    NOTE:
    Semideviation is the std dev of negative returns
    r must be a Series or a DataFrame, else raises a TypeError
    """
    semid_var_cvar_df = pd.DataFrame()
    hist_var = []
    for returns_df in returns.values():
        semidev = returns_df[returns_df<0].std(ddof=0)
        hist_var.append(np.percentile(returns_df, worst_percent)*-1)
        z = stats.norm.ppf(worst_percent/100)
        gaus_var = (returns_df.mean() + z*returns_df.std(ddof=0))*-1

        ## calculate skewness, kurtosis and Cornish-Fisher VAR
        demeaned_ret = returns_df-returns_df.mean()
        stdev = returns_df.std(ddof=0)      ##Delta Degrees of Freedom. The divisor used in calculations is N - ddof, where N represents the number of elements.
        skewness = ((demeaned_ret**3).mean())/(stdev**3)
        kurtosis = ((demeaned_ret**4).mean())/(stdev**4)
        cf_z = (z + (z**2 - 1)*skewness/6 +(z**3 -3*z)*(kurtosis-3)/24 -(2*z**3 - 5*z)*(skewness**2)/36)
        cf_var = -(returns_df.mean() + cf_z*stdev)
        
        #Calculate BeyondVaR or CVaR
        returns_beyond = returns_df <= (np.percentile(returns_df, 5))
        cvar = -(returns_df[returns_beyond].mean())
        
        semid_var_cvar_df = pd.concat([semid_var_cvar_df,pd.concat([semidev, gaus_var, cf_var, cvar], axis="columns")],axis=0)
    semid_var_cvar_df.columns = ['Semi-Deviation', 'Gaussian VaR', 'Cornish-Fisher VaR','Conditional VaR']
    semid_var_cvar_df['Historical VaR'] = hist_var
    return semid_var_cvar_df

In [41]:
get_semideviation_var_cvar(returns, worst_percent=5)

,Semi-Deviation,Gaussian VaR,Cornish-Fisher VaR,Conditional VaR,Historical VaR
AVANTIFEED,0.042588,0.070323,0.004841,0.085836,0.043719
BAJAJFINSV,0.015653,0.036830,0.031059,0.048798,0.031706
BAJAJHLDNG,0.014044,0.031282,0.025132,0.042701,0.026156
BAJFINANCE,0.029288,0.048373,-0.083075,0.058301,0.033738


# Covariance Matrix

In [1]:
def get_covariance_matrix(returns):
    combined_df = pd.DataFrame(index=list(returns.values())[0].index)
    for df in returns.values():
         combined_df = pd.concat([combined_df.loc[~combined_df.index.duplicated(keep='first')],df.loc[~df.index.duplicated(keep='first')]], join='inner', axis=1, sort=False)
    return combined_df.cov()

# Timeseries length comparison

In [2]:
def compare_timeseries_lengths(returns):
    start = []
    end = []
    length = []
    for df in returns.values():
        start.append(list(df.index)[0])
        end.append(list(df.index)[-1])
        length.append(len(list(df.index)))
    dates = pd.DataFrame({'Start Date':start,'End Date':end, 'Length': length}, index=["BAJAJFINSV","BAJAJHLDNG","BAJFINANCE","BATAINDIA","BERGEPAINT","BRITANNIA","EICHERMOT","HDFCBANK","HINDUNILVR","ICICIBANK","JUBLFOOD","M&M","MOTHERSUMI","NCC","OLECTRA","RELIANCE","PIDILITIND","SBIN","SOLARINDS","TATACONSUM","TRENT"])
    return dates

# Efficient Frontier

In [4]:
def get_portfolio_return(weights, annualized_returns):
    '''
    INPUT: weights and the annualized returns of all companies as a series.
    weights are a numpy array or Nx1 matrix and returns are a numpy array or Nx1 matrix/dataframe
    
    OUTPUT: weighted return of the portfolio
    '''
    return weights.T @ annualized_returns

In [5]:
def get_portfolio_vol(weights, cov_matrix):
    """
    INPUT: weights are a numpy array or N x 1 maxtrix and covmat is an N x N matrix/dataframe
    
    OUTPUT: the volatility of the portfolio.
    
    Computes the vol of a portfolio from a covariance matrix and constituent weights
    """
    return (weights.T @ cov_matrix @ weights)**0.5

In [9]:
def get_efficient_frontier_2assets(returns_2, points_on_frontier=20):
    """
    INPUT: the return dictionary of the 2 assets in the portfolio, 
    the number of points on the frontier for which you wish to calculate the return and risk
    
    OUTPUT: the 2-asset efficient frontier
    """
    weights_2 = [np.array([w, 1-w]) for w in np.linspace(0,1,points_on_frontier)]
    eff_frontier = pd.DataFrame(weights_2, columns=['Wt of '+x for x in list(returns_2.keys())])
    portfolio_return = []
    portfolio_risk = []
    ann_returns_2 = get_AnnVol_AnnRet_Ret2Risk_Sharpe(returns_2)['Annualized Returns']
    combined_returns_df, cov_matrix_2 = get_covariance_matrix(returns_2)
    for weights in weights_2:
        portfolio_return.append(get_portfolio_return(weights, ann_returns_2))
        portfolio_risk.append(get_portfolio_vol(weights, cov_matrix_2))
    eff_frontier['Portfolio Return'], eff_frontier['Portfolio Volatility'] = [portfolio_return, portfolio_risk]
    return eff_frontier

In [1]:
def minimize_vol(returns, target_return):
    """
    INPUT: returns of assets in portfolio
    
    OUTPUT: The weights for a portfolio consisting of
    the input assets in 'returns' that give the target return
    """
    n_assets = len(list(returns.keys()))
    init_guess = np.repeat(1/n_assets, n_assets)
    bound = ((0.0,1.0),)*n_assets
    
    ann_returns = get_AnnVol_AnnRet_Ret2Risk_Sharpe(returns)['Annualized Returns']
    combined_returns_df, cov_matrix = get_covariance_matrix(returns)
    
    return_is_target = {
        'type': 'eq',
        'args': (ann_returns,),
        'fun': lambda weights, ann_returns: target_return - get_portfolio_return(weights, ann_returns)
    }
    weights_sum_to_1 = {
        'type': 'eq',
        'fun': lambda weights: np.sum(weights) - 1
    }
    
    results = minimize(get_portfolio_vol, init_guess,
                      args=(cov_matrix,), method='SLSQP',
                       options={'disp': False},
                       constraints=(weights_sum_to_1,return_is_target),
                       bounds=bound)
    
    return results.x

In [2]:
def get_frontier_weights(returns, num_target_points=50):
    """
    INPUT: returns of assets to be used in portfolio and number of
    points for which you need the weights
    
    OUTPUT: gives a list of returns between the min and max 
    annualised returns among the assets that are to be included
    in the portfolio
    """
    annualized_return = get_AnnVol_AnnRet_Ret2Risk_Sharpe(returns)['Annualized Returns']
    frontier_target_returns = np.linspace(annualized_return.min(), annualized_return.max(), num_target_points)
    frontier_target_weights = [minimize_vol(returns, target_return) for target_return in frontier_target_returns]
    return frontier_target_weights

In [3]:
def plot_efficient_frontier(returns, num_of_front_pts=50):
    """
    Plots the multi-asset efficient frontier
    
    INPUT: returns of assets in portfolio and number of points on
    the frontier that you wish to plot
    
    OUTPUT: risk vs return plot 
    """
    optimal_weights = get_frontier_weights(returns, num_of_front_pts)
    annualized_return = get_AnnVol_AnnRet_Ret2Risk_Sharpe(returns)['Annualized Returns']
    combined_returns_df, cov_matrix = get_covariance_matrix(returns)
    rets = [get_portfolio_return(w, annualized_return) for w in optimal_weights]
    vols = [get_portfolio_vol(w, cov_matrix) for w in optimal_weights]
    ef = pd.DataFrame({
        "Returns": rets, 
        "Volatility": vols
    })
    return ef.plot.line(x="Volatility", y="Returns", style='.-')